# Pergunta a ser respondida:

- Classificar se um indivíduo sobreviveu ou morreu ao afundamento do Titanic.

  - Temos uma pergunta de classificação, pois estamos fazendo a predição de um rótulo (label) acerca da sobrevivência: um passageiro sobreviveu ou morreu.

## Dicionários dos atributos:

- pclass: classe do passageiro (1=primeiroa, 2=segunda, 3=terceira);
- survived: sobreviveu (0=não, 1=sim);
- name: nome;
- sex: sexo;
- age: idade;
- sibsp: número de irmãos/esposa(o) a bordo;
- parch: número de pais/filhos a bordo;
- ticket: número da passagem;
- fare: preço da passagem;
- cabin: cabine;
- embarked: local em que o passageiro embarcou (C=Cherbourg, Q=Queenstown, S=Southampton);
- boat: bote salva-vidas;
- body: número de identificação do corpo;
- home.dest: lar/destino;

## Refatoração do código

### Imports necessários

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.classifier import ROCAUC
from sklearn import (
    ensemble,
    preprocessing,
    tree
)
from sklearn.metrics import (
    auc,
    confusion_matrix,
    roc_auc_score,
    roc_curve
)
from sklearn import model_selection
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold
)
from yellowbrick.model_selection import (
    LearningCurve
)
from sklearn.experimental import (enable_iterative_imputer)
from sklearn import impute

### Extração dos dados e formação do dataframe

In [2]:
df = pd.read_csv("titanic3.csv")

## Funções:

### Limpeza dos dados

- exclusão dos campos desnecessários;
- transformação das variáveis dummies;

In [3]:
def limpeza_dados(df):
  df = df.drop(
            columns=[
                      "name",
                      "ticket",
                      "home.dest",
                      "boat",
                      "body",
                      "cabin"
  ]).pipe(pd.get_dummies, drop_first=True) # O uso do pipe permite que essas operações sejam encadeadas de forma mais limpa e eficiente

  return df

### Divisão de dados para treino e teste e imputando dados ausentes

In [4]:
def get_train_test_X_y(df, y_col, size=0.3, std_cols=None):
  y = df[y_col]
  X = df.drop(columns=y_col)
  X_train, X_test, y_train, y_test = model_selection.train_test_split(
                                                                      X, y, test_size=size, random_state=42
                                                                    )
  cols = X.columns
  num_cols = [
                "pclass",
                "age",
                "sibsp",
                "parch",
                "fare"
              ]

  fi = impute.IterativeImputer()
  X_train.loc[:, num_cols] = fi.fit_transform(X_train[num_cols])
  X_test.loc[:, num_cols] = fi.transform(X_test[num_cols])

  if std_cols:
    std = preprocessing.StandardScaler()
    X_train.loc[:, std_cols] = std.fit_transform(X_train[std_cols])
    X_test.loc[:, std_cols] = std.transform(X_test[std_cols])

    return X_train, X_test, y_train, y_test

#.loc[:, num_cols] - pega todas as linhas (:) e as coliunas indicadas na variável num_cols

## Chamando as funções

In [5]:
ti_df = limpeza_dados(df)
std_cols = "pclass,age,sibsp,fare".split(",")
X_train, X_test, y_train, y_test = get_train_test_X_y(ti_df, "survived", std_cols=std_cols)

## Modelo - teste 1

In [6]:
from sklearn.dummy import DummyClassifier

bm = DummyClassifier()

bm.fit(X_train, y_train)
bm.score(X_test, y_test) # precisão

0.5699745547073791

### Vendo a métrica da precisão

In [7]:
from sklearn import metrics

metrics.precision_score(y_test, bm.predict(X_test))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0

## Testando outros modelos para comparação

- Objetivo: encontrar um modelo que tenha uma pontuação média um pouco menor e um desvio-padrão um pouco menor.

In [8]:
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

In [12]:
from sklearn import model_selection
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost

In [15]:
for model in [
                DummyClassifier,
                LogisticRegression,
                DecisionTreeClassifier,
                KNeighborsClassifier,
                GaussianNB,
                SVC,
                RandomForestClassifier,
                xgboost.XGBClassifier
                ]:
  cls = model()
  kfold = model_selection.KFold(n_splits=10, random_state=42, shuffle=True)
  s = model_selection.cross_val_score(cls, X, y, scoring="roc_auc", cv=kfold)
  print (
          f"{model.__name__:22} AUC: "
          f"{s.mean():.3f} STD: {s.std():.2f}"
  )

DummyClassifier        AUC: 0.500 STD: 0.00
LogisticRegression     AUC: 0.840 STD: 0.04
DecisionTreeClassifier AUC: 0.768 STD: 0.02
KNeighborsClassifier   AUC: 0.829 STD: 0.02
GaussianNB             AUC: 0.811 STD: 0.05
SVC                    AUC: 0.838 STD: 0.03
RandomForestClassifier AUC: 0.849 STD: 0.02
XGBClassifier          AUC: 0.853 STD: 0.02


### Definições:

- **Pontuação AUC**
AUC (Area Under the Curve) é uma métrica comum usada para **avaliar a performance de modelos de classificação binária**.

  - O que é AUC?

Definição: A AUC mede a área sob a curva ROC (Receiver Operating Characteristic). A curva ROC é uma representação gráfica que mostra o desempenho de um modelo de classificação binária em diferentes thresholds de discriminação.

**Valores: A AUC varia de 0 a 1**, onde:

    - Um AUC de 1 indica um modelo perfeito, que é capaz de fazer todas as previsões corretas.
    - Um AUC de 0.5 indica um modelo que faz previsões aleatórias, sem discriminação entre as classes.

 - Como a AUC é calculada?
A AUC é calculada traçando a curva ROC e calculando a área sob essa curva. Aqui estão os passos principais:

    - Curva ROC: A curva ROC é construída traçando a Taxa de Verdadeiros Positivos (True Positive Rate, TPR) contra a Taxa de Falsos Positivos (False Positive Rate, FPR) para diferentes valores de threshold de classificação.

    - TPR (Sensibilidade): É a proporção de exemplos positivos que são corretamente classificados como positivos.

    - FPR: É a proporção de exemplos negativos que são incorretamente classificados como positivos.

    - Área sob a curva ROC: A AUC é então calculada como a área sob essa curva ROC. Quanto maior a área sob a curva, melhor é o desempenho do modelo em distinguir entre as classes.

  - Quando usar a AUC?
A **AUC é especialmente útil quando você tem um conjunto de dados desbalanceado**, ou seja, quando uma classe é muito mais prevalente do que a outra. Nesses casos, a precisão e a acurácia podem não ser métricas adequadas, pois podem ser enganosamente altas apenas devido ao desbalanceamento.

Além disso, a **AUC é uma métrica robusta que não é afetada por mudanças na proporção de classes ou na distribuição dos dados.**

**Threshold" (limiar)** é usado em contextos de classificação para **definir um ponto de corte ou um valor de referência para distinguir entre as classes positiva e negativa.**

- Definição: Um threshold **é um valor numérico que define a fronteira entre duas classes em um problema de classificação binária.** Ele determina a decisão de como as previsões do modelo são interpretadas como pertencentes à classe positiva ou à classe negativa.

- Aplicação: Em um modelo de classificação binária, após fazer previsões probabilísticas (por exemplo, probabilidade de um exemplo pertencer à classe positiva), você pode aplicar um threshold para converter essas probabilidades em rótulos de classe.

- Decisão com Threshold: Para transformar essas probabilidades em rótulos de classe, você aplica um threshold. Por exemplo, se o threshold for 0.5, todas as previsões com probabilidades maiores ou iguais a 0.5 serão rotuladas como classe positiva, e as previsões com probabilidades menores que 0.5 serão rotuladas como classe negativa.

- Importância do Threshold:
  - Equilíbrio de Precisão e Recall: O threshold permite ajustar o equilíbrio entre precisão e recall do modelo. Por exemplo, um threshold mais baixo pode aumentar o recall (capturando mais exemplos positivos), mas diminuir a precisão (podendo aumentar o número de falsos positivos). Um threshold mais alto pode aumentar a precisão (reduzindo os falsos positivos), mas diminuir o recall (podendo perder alguns exemplos positivos).

  - Aplicação Contextual: O threshold deve ser escolhido com base no contexto do problema e nos requisitos específicos de negócio. Por exemplo, em um problema médico de detecção de doenças, pode ser mais importante maximizar o recall, mesmo que isso signifique ter mais falsos positivos.

- Como Escolher um Threshold?
  - Análise da Curva ROC: A curva ROC mostra a relação entre a taxa de verdadeiros positivos (TPR) e a taxa de falsos positivos (FPR) para diferentes thresholds. Escolher um threshold adequado muitas vezes envolve analisar essa curva e escolher um ponto que atenda aos requisitos do seu problema.

  - F1-score ou outras métricas: Além da AUC, métricas como F1-score (que combina precisão e recall) podem ser usadas para ajudar a escolher um threshold.

  - Requisitos de Negócio: Em muitos casos, os requisitos específicos do negócio ou da aplicação determinarão qual é o threshold ideal. Por exemplo, se um falso positivo é muito caro em termos de recursos ou impacto, você pode escolher um threshold mais alto para reduzir esses casos.